### 1. 요리 재료 전처리  & xlsx -> csv

In [4]:
import pandas as pd

# 1. 파일 이름 정의
folder = '../data/'
file_in = folder + 'recipe_ingredient.xlsx'
file_out = folder + 'recipe_ingredient_cleaned.csv'

try:
    # 2. 파일 로드 (한글 깨짐 및 오류 방지)
    try:
        df = pd.read_excel(file_in)
    except UnicodeDecodeError:
        print("cp949 인코딩 실패, utf-8로 재시도...")
        df = pd.read_excel(file_in, encoding='utf-8')
    except Exception as e:
        print(f"읽기 오류 발생. Python 엔진으로 재시도...: {e}")
        df = pd.read_excel(file_in, encoding='cp949', engine='python', on_bad_lines='skip')


    # --- (핵심) 3. 사용자 정의 정렬 순서 생성 ---
    # '주재료' 1순위, '부재료' 2순위, '양념' 3순위로 지정
    custom_order = {
        '주재료': 1,
        '부재료': 2,
        '양념': 3
    }
    
    # '재료타입명'을 기준으로 새로운 정렬용 'sort_order' 열을 생성합니다.
    # 목록에 없는 값은 99순위로 밀려납니다.
    df['sort_order'] = df['재료타입명'].map(custom_order).fillna(99)

    # --- 4. 데이터 정렬 ---
    # 1순위: 방금 만든 'sort_order' (주재료 > 부재료 > 양념)
    # 2순위: 레시피 코드
    # 3순위: 재료순번
    df_sorted = df.sort_values(by=['sort_order', '레시피 코드', '재료순번'])

    # 5. 불필요한 열 제거
    # 정렬에 사용한 'sort_order' 열은 삭제합니다.
    df_sorted = df_sorted.drop(columns=['sort_order'])

    # 6. 파일 저장
    # 엑셀 호환성을 위해 'utf-8-sig'로 저장
    df_sorted.to_csv(file_out, index=False, encoding='utf-8-sig')

    print(f"파일 처리 완료! 정렬된 파일이 '{file_out}'로 저장되었습니다.")
    print("\n정렬된 데이터 상위 5개 행:")
    print(df_sorted.head())

except FileNotFoundError:
    print(f"오류: '{file_in}' 파일을 찾을 수 없습니다.")
except Exception as e:
    print(f"오류 발생: {e}")

파일 처리 완료! 정렬된 파일이 '../data/recipe_ingredient_cleaned.csv'로 저장되었습니다.

정렬된 데이터 상위 5개 행:
   레시피 코드  재료순번  재료명  재료용량  재료타입 코드 재료타입명
0       1     1    쌀    4컵  3060001   주재료
1       1     2   안심  200g  3060001   주재료
2       1     3  콩나물   20g  3060001   주재료
3       1     4  청포묵  1/2모  3060001   주재료
4       1     5  미나리   20g  3060001   주재료
